# `Automated script to download Sentinel-1 images using Alaska Satellite Facility API`


# Import libraries and packages

In [1]:
!pip install asf_search
!pip install shapely
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 10.5 MB/s eta 0:00:00


In [14]:
import asf_search as asf
from shapely.geometry import box, mapping
from datetime import datetime, timezone
import os
import zipfile
import json
import rasterio
import rasterio.mask
from scipy.signal import convolve2d
from rasterio import shutil as rio_shutil
from google.colab import files
import getpass
import numpy as np

# Authenticate with Earthdata NASA

In [3]:
# --- Prompt for Earthdata credentials securely ---
username = input("Enter your Earthdata username: ")
password = getpass.getpass("Enter your Earthdata password: ")

# --- Authenticate with Earthdata credentials ---
session = asf.ASFSession().auth_with_creds(username, password)

Enter your Earthdata username: carlosmendez1997
Enter your Earthdata password: ··········


# Define Area of Interest (AOI)

In [15]:
# --- Define bounding box coordinates ---
xmin, ymin, xmax, ymax = -68.803909, -11.033737, -68.725933, -10.979773

# --- Create shapely polygon and convert to WKT ---
bbox_geom = box(xmin, ymin, xmax, ymax)
bbox_wkt = bbox_geom.wkt

# --- Convert shapely polygon to GeoJSON for rasterio.mask ---
geojson_geom = [mapping(bbox_geom)]

# Define search parameters

In [5]:
# --- Define search parameters ---
platform = "Sentinel-1A" # Satellite platform
polarizations = ["VV", "VH"] # Polarizations to include
beam_mode = "IW" # Beam mode
processing_level = "GRD" # Processing level
orbit_direction = "DESCENDING"
relative_orbit = 127 # Relative orbit number

# Set date range

In [6]:
# --- Date range ---
start_date = "2026-01-01" # Start date for search
end_date = datetime.now(timezone.utc).strftime("%Y-%m-%d") # Current date as end

# Perform Search of S1

In [7]:
# --- Perform search ---
results = asf.search(
    platform=platform,
    processingLevel=processing_level,
    beamMode=beam_mode,
    polarization=polarizations,
    relativeOrbit=relative_orbit,
    start=start_date,
    end=end_date,
    intersectsWith=bbox_wkt
)

# --- Print number of scenes found ---
print(f"Found {len(results)} Sentinel-1 scenes")

Found 16 Sentinel-1 scenes


# Create folder to dowload data

In [8]:
# --- Create folder for downloads ---
os.makedirs("ASF_Sentinel1", exist_ok=True)

# Download results

In [9]:
# --- Download results into folder ---
results.download(path="ASF_Sentinel1", session=session)

/usr/local/lib/python3.12/dist-packages/asf_search/download/download.py:66: UserWarning: File already exists, skipping download: ASF_Sentinel1/OPERA_L2_RTC-S1_T127-271782-IW1_20260207T101341Z_20260207T222557Z_S1C_30_v1.0.h5
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')
/usr/local/lib/python3.12/dist-packages/asf_search/download/download.py:66: UserWarning: File already exists, skipping download: ASF_Sentinel1/OPERA_L2_RTC-S1_T127-271782-IW1_20260126T101341Z_20260126T191435Z_S1C_30_v1.0.h5
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')
/usr/local/lib/python3.12/dist-packages/asf_search/download/download.py:66: UserWarning: File already exists, skipping download: ASF_Sentinel1/OPERA_L2_RTC-S1_T127-271782-IW1_20260108T101442Z_20260108T164159Z_S1A_30_v1.0.h5
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')
/usr/local/lib/python3.12/dist-packages/asf_search/down

# Create output folder for GeoTIFFs

In [10]:
# --- Create output folder for GeoTIFFs ---
output_dir = "ASF_Sentinel1/GeoTIFFs"
os.makedirs(output_dir, exist_ok=True)

In [17]:
with rasterio.open(src_path) as src:
    print("Raster bounds:", src.bounds)
    print("Raster CRS:", src.crs)
    print("AOI bounds:", bbox_geom.bounds)

Raster bounds: BoundingBox(left=0.0, bottom=1491.0, right=21359.0, top=0.0)
Raster CRS: None
AOI bounds: (-68.803909, -11.033737, -68.725933, -10.979773)


In [18]:
gdalinfo /content/ASF_Sentinel1/S1_271782_IW1_20260102T101342_VH_CB95-BURST.tiff

NameError: name 'gdalinfo' is not defined

# Additional processing S1 images

In [16]:
# --- Function to clip raster using rasterio.mask ---
def clip_raster(src_path, out_path, geojson_geom):
    with rasterio.open(src_path) as src:
        clipped_data, clipped_transform = rasterio.mask.mask(src, geojson_geom, crop=True)
        profile = src.profile
        profile.update({
            "height": clipped_data.shape[1],
            "width": clipped_data.shape[2],
            "transform": clipped_transform
        })
        with rasterio.open(out_path, "w", **profile) as dst:
            dst.write(clipped_data)

# --- Simple speckle reduction filter (mean filter) ---
def speckle_filter(src_path, out_path):
    with rasterio.open(src_path) as src:
        data = src.read(1)  # Read first band
        # Apply simple 3x3 mean filter
        kernel = np.ones((3,3)) / 9.0
        filtered = convolve2d(data, kernel, mode="same", boundary="symm")
        profile = src.profile
        with rasterio.open(out_path, "w", **profile) as dst:
            dst.write(filtered.astype(profile["dtype"]), 1)

# --- Filter and process only .tiff files (VV/VH) ---
for root, dirs, files_in_dir in os.walk("ASF_Sentinel1"):
    for f in files_in_dir:
        if f.endswith(".tiff") and ("VV" in f or "VH" in f):
            src_path = os.path.join(root, f)
            try:
                date_str = f.split("_")[3][:8]  # Extract date (YYYYMMDD)
            except Exception:
                date_str = "unknown"
            pol = "VV" if "VV" in f else "VH"
            out_name = f"S1_{pol}_{date_str}.tif"
            out_path = os.path.join(output_dir, out_name)

            # Clip raster to bounding box
            clip_raster(src_path, out_path, geojson_geom)
            print(f"Saved clipped raster: {out_path}")

            # Apply speckle filter
            filtered_path = out_path.replace(".tif", "_filtered.tif")
            speckle_filter(out_path, filtered_path)
            print(f"Saved speckle-filtered raster: {filtered_path}")

ValueError: Input shapes do not overlap raster.

# Compress all GeoTIFFs into a single ZIP

In [11]:
# --- Compress all GeoTIFFs into a single ZIP ---
zip_path = "ASF_Sentinel1/Sentinel1_GeoTIFFs.zip"
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for file in os.listdir(output_dir):
        file_path = os.path.join(output_dir, file)
        zipf.write(file_path, arcname=file)

print(f"✅ Compressed file ready: {zip_path}")

✅ Compressed file ready: ASF_Sentinel1/Sentinel1_GeoTIFFs.zip


# Download ZIP to local machine

In [12]:
# --- Download ZIP to local machine ---
files.download(zip_path)

AttributeError: 'list' object has no attribute 'download'